In [1]:
try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen
    
from bs4 import BeautifulSoup
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
import re
import pandas as pd
from time import sleep
from selenium import webdriver
import requests
import time
import sys
from requests.exceptions import HTTPError
from selenium.webdriver.chrome.options import Options

In [2]:
t0 = time.time()

#path_chrome = "C:/Program Files/Google/Chrome/Application/chrome.exe"
#path_chrome_driver = "C:/Users/anhnt/Downloads/chromedriver_win32/chromedriver.exe"

#path_chrome = "C:/Program Files (x86)/Google/Chrome/Application/chrome.exe"
#path_chrome_driver = "E:/ERP_APP/APP_VTSTORE_COMPARE_PRICE_LIVE/Tool/Setup_SSG/chromedriver_win32/chromedriver.exe"
    
path_chrome = "C:/Program Files (x86)/Google/Chrome/Application/chrome.exe",
path_chrome_driver = "C:/Program Files (x86)/Google/chromedriver_win32/chromedriver.exe",
timeout = 40
    
def get_obj(url):
    html = urlopen(url)
    return BeautifulSoup(html, "lxml")

def get_obj2(url):
    response = requests.get(url)
    return BeautifulSoup(response.text,"lxml")
    
def link_fpt_dien_thoai_all():
    link_fpt_dien_thoai = "https://fptshop.com.vn/dien-thoai?sort=gia-cao-den-thap"
    urls = []
    for i in range(16):
        if i ==0:
            url = link_fpt_dien_thoai
        else:
            url = link_fpt_dien_thoai+"&trang=" + str(i)
        urls.append(url)
    return urls

def to_number(df):
    for i in range(len(df)):
        df.loc[i, 'Price'] = df.loc[i, 'Price'].replace('.', '')
        df.loc[i, 'Price'] = df.loc[i, 'Price'].replace(',', '')
        df.loc[i, 'Price'] = df.loc[i, 'Price'].replace(' ₫', '')
        df.loc[i, 'Price'] = df.loc[i, 'Price'].replace('đ', '')
        df.loc[i, 'Price'] = df.loc[i, 'Price'].replace('\xe2\x82\xab', '')
        df.loc[i, 'Name'] = df.loc[i, 'Name'].replace('Điện thoại ', '')
        df.loc[i, 'Promotion'] = df.loc[i, 'Promotion'].replace('Xem chi tiết', '')
        df.loc[i, 'Promotion'] = " ".join(df.loc[i, 'Promotion'].split())
    return df

In [3]:
link_vts_dien_thoai = 'https://viettelstore.vn/danh-muc/dien-thoai-010001.html'
chrome_driver_binary = path_chrome_driver

def links_dienthoai_vts(url):
    for t in range(2):
        for i in range(3):
            try:
                driver = webdriver.Chrome(chrome_driver_binary)
                driver.set_page_load_timeout(timeout)
                driver.get(url)
                break
            except:
                print("Timeout Exception has been thrown: " + str(i))
                driver.quit()
                sleep(5)
                continue

        num = 0

        for i in range(12):
            for elem in driver.find_elements_by_css_selector("#div_Danh_Sach_San_Pham_loadMore_btn > a"):
                driver.execute_script("arguments[0].click();", elem)
                sleep(4)

        # Selenium hands the page source to Beautiful Soup
        vts = BeautifulSoup(driver.page_source, 'lxml')

        links = []
        allPhone = vts.findAll('a', href  = re.compile("^(/dien-thoai/)"))
        for phone in allPhone:
            link = 'https://viettelstore.vn' + phone.attrs['href']
            links.append(link)
        if (len(links) > 0):
            break
    driver.close()
    driver.quit()
    return links

link_vts = []
nums = []

link = links_dienthoai_vts(link_vts_dien_thoai)
nums.append(len(link))
link_vts.append(link)

In [16]:
for (i, u) in enumerate(link_vts[nums.index(max(nums))]):
    print i, u

0 https://viettelstore.vn/dien-thoai/vivo-y81i-pid131985.html
1 https://viettelstore.vn/dien-thoai/oppo-a3s-pid128800.html
2 https://viettelstore.vn/dien-thoai/samsung-a7-2018-pid130151.html
3 https://viettelstore.vn/dien-thoai/samsung-galaxy-a9-2018-pid132063.html
4 https://viettelstore.vn/dien-thoai/samsung-galaxy-m20-pid133349.html
5 https://viettelstore.vn/dien-thoai/nokia-8-1-pid133458.html
6 https://viettelstore.vn/dien-thoai/oppo-f9-pid128987.html
7 https://viettelstore.vn/dien-thoai/oppo-a7-pid131993.html
8 https://viettelstore.vn/dien-thoai/huawei-nova-3i-pid128371.html
9 https://viettelstore.vn/dien-thoai/huawei-y7-pro-2019-pid133439.html
10 https://viettelstore.vn/dien-thoai/huawei-y9-2019-pid131480.html
11 https://viettelstore.vn/dien-thoai/huawei-mate-20-pro-pid131840.html
12 https://viettelstore.vn/dien-thoai/huawei-mate-20-pid131839.html
13 https://viettelstore.vn/dien-thoai/iphone-6s-plus-32gb-pid119607.html
14 https://viettelstore.vn/dien-thoai/iphone-x-64gb-pid118486.

In [19]:
df_vts = pd.DataFrame(columns=['Name', 'Price', 'Promotion'])
list_exception = []

def get_info_vts(urls):
    names = []
    prices = []
    KMs = []
   
    for (i, u) in enumerate(urls):       
        driver = webdriver.Chrome(chrome_driver_binary)
        driver.set_page_load_timeout(timeout)
        try:
            driver.get(u)
        except:
            print("Timeout Exception, " + str(i) + ": " + str(u))
            list_exception.append(u)
            driver.close()
            continue
            
        content = driver.page_source.encode('utf-8').strip()
        soup = BeautifulSoup(content,"html.parser")
        
        name = soup.find('h1', {'class':'txt-24'})
        price = soup.find('span', {'id':'_price_new436'})
        KM = soup.find('div', {'class':'body-promotion'})
        if (name  is not None):
            n = name.get_text().encode('utf-8').strip()
        else:
            n = ''
        if (price  is not None):
            p = price.get_text().encode('utf-8').strip()
        else:
            p = ''
        if (KM  is not None):
            pr = KM.get_text().encode('utf-8').strip()
        else:
            pr = ''
        names.append(n)
        prices.append(p)
        KMs.append(pr)
        
        driver.close()
        driver.quit()
        sleep(0.3)
    return names, prices, KMs

names, prices, KMs = get_info_vts(link_vts[nums.index(max(nums))])

for i in range(len(names)):
    df_vts.loc[i, 'Name'] = names[i].lower().strip()
    df_vts.loc[i, 'Price'] = prices[i]
    df_vts.loc[i, 'Promotion'] = KMs[i].strip()
    df_vts.loc[i, 'Name'] = df_vts.loc[i, 'Name'].replace('Điện thoại ', '')

df_vts['Maingroup'] = 'dienthoai'

to_number(df_vts)

Timeout Exception has been thrown. 30: https://viettelstore.vn/dien-thoai/vivo-v9-youth-pid126405.html
Timeout Exception has been thrown. 63: https://viettelstore.vn/dien-thoai/symphony-b01-pid133181.html
Timeout Exception has been thrown. 70: https://viettelstore.vn/dien-thoai/symphony-v130-pid133055.html
Timeout Exception has been thrown. 71: https://viettelstore.vn/dien-thoai/nokia-1-pid125080.html
Timeout Exception has been thrown. 72: https://viettelstore.vn/dien-thoai/masstel-play-30-pid133009.html
Timeout Exception has been thrown. 107: https://viettelstore.vn/dien-thoai/oppo-find-x-pid128494.html
Timeout Exception has been thrown. 108: https://viettelstore.vn/dien-thoai/masstel-juno-q3-pid129921.html
Timeout Exception has been thrown. 110: https://viettelstore.vn/dien-thoai/samsung-galaxy-a8-star-2018-pid126894.html
Timeout Exception has been thrown. 112: https://viettelstore.vn/dien-thoai/itel-2123-pid121019.html
Timeout Exception has been thrown. 114: https://viettelstore.vn/

,Name,Price,Promotion,Maingroup
0,vivo y81i,2990000,Tặng Phiếu mua hàng 100.000đ Cơ hội rước H...,dienthoai
1,oppo a3s 16gb,3290000,Trả góp 0% (làm hợp đồng trên giá bán lẻ) ...,dienthoai
2,samsung galaxy a7 2018,5990000,Trả góp 0% trên giá giảm Cơ hội rước Heo V...,dienthoai
3,samsung galaxy a9 (2018),10490000,"Trả góp 0% trên giá giảm Giảm thêm 500,000...",dienthoai
4,samsung galaxy m20,4990000,Trả góp 0% Tặng Phiếu mua hàng 100.000đ Ho...,dienthoai
5,nokia 8.1,7990000,Lì xì 1 thùng Coca (24 lon) Tặng sạc dự ph...,dienthoai
6,oppo f9,6990000,Trả góp 0% (làm HĐ trên giá bán lẻ) Lì xì ...,dienthoai
7,oppo a7 64gb,5990000,Trả góp 0% (làm HĐ trên giá bán lẻ) Lì xì ...,dienthoai
8,huawei nova 3i,6490000,Trả góp 0% Lì xì Phiếu mua hàng 300.000đ ...,dienthoai
9,huawei y7 pro (2019),3990000,Trả góp lãi suất 0% Lì xì Phiếu mua hàng 8...,dienthoai


In [6]:
'''
print list_exception
import xlsxwriter

# Create a workbook and add a worksheet.
filename = time.strftime("%Y%m%d-%H%M%S-exception") + '.xlsx'

workbook = xlsxwriter.Workbook(filename)
worksheet = workbook.add_worksheet('vts')

writer = pd.ExcelWriter(filename, engine='openpyxl')
df_fpt.to_excel(writer,'vts',index=False)
'''

'\nprint list_exception\nimport xlsxwriter\n\n# Create a workbook and add a worksheet.\nfilename = time.strftime("%Y%m%d-%H%M%S-exception") + \'.xlsx\'\n\nworkbook = xlsxwriter.Workbook(filename)\nworksheet = workbook.add_worksheet(\'vts\')\n\nwriter = pd.ExcelWriter(filename, engine=\'openpyxl\')\ndf_fpt.to_excel(writer,\'vts\',index=False)\n'

In [7]:
### SCRAPE FPTSHOP - DIENTHOAI
link_fpt_dien_thoai = "https://fptshop.com.vn/dien-thoai?sort=gia-cao-den-thap"

names = []
prices = []
KMs = []
num = 0
df_fpt = pd.DataFrame(columns=['Name', 'Price', 'Promotion'])

try:
    for url in link_fpt_dien_thoai_all():
        fpt_obj = get_obj(url)

        allPhoneInUrl = fpt_obj.findAll('div', {'class':'fs-lpil-if'})
        for s in allPhoneInUrl:
            price = s.find('div',{'class':'fs-lpilname'}).find("div", {"class":"fs-lpil-price"}).find('p')
            name = s.find('div',{'class':'fs-lpilname'}).find("h3", {"class":"fs-lpil-name"}).find('a')
            KM = s.find('div', {'class':'fs-icpmbox'})

            if (name is not None):
                n = name.get_text().encode('utf-8')
                names.append(n)
            else:
                n = ''
                names.append(n)

            if (price is not None):
                p = price.get_text().encode('utf-8')
                prices.append(p)
            else:
                p = ''
                prices.append(p)     

            if (KM is not None): 
                k = KM.get_text().encode('utf-8').rstrip()
                KMs.append(k)
            else:
                k = ''
                KMs.append(k)

            num += 1
            
    for i in range(len(names)):
        df_fpt.loc[i, 'Name'] = names[i].lower().strip()
        df_fpt.loc[i, 'Price'] = prices[i]
        df_fpt.loc[i, 'Promotion'] = KMs[i].strip()
        df_fpt.loc[i, 'Name'] = df_fpt.loc[i, 'Name'].replace('Điện thoại ', '')
    
    df_fpt['Maingroup'] = 'dienthoai'
except:
    print 'Could not open FPTShop'
    
to_number(df_fpt)

,Name,Price,Promotion,Maingroup
0,iphone xs max 512gb,43990000,Khách hàng chọn 1 trong 2 khuyến mại sau: KM1:...,dienthoai
1,iphone xs 512gb,40990000,Khách hàng chọn 1 trong 2 khuyến mại sau:KM1:T...,dienthoai
2,iphone xs max 256gb,37990000,Khách hàng chọn 1 trong 2 khuyến mại sau: KM1:...,dienthoai
3,iphone xs 256gb,34990000,Khách hàng chọn 1 trong 2 khuyến mại sau: KM1:...,dienthoai
4,iphone xs max 64gb,33990000,Khách hàng chọn 1 trong 2 khuyến mại sau: KM1:...,dienthoai
5,iphone xs 64gb,29990000,Khách hàng chọn 1 trong 2 khuyến mại sau: KM1:...,dienthoai
6,iphone xr 256gb,24990000,Khách hàng chọn 1 trong 2 khuyến mại sau:KM1:T...,dienthoai
7,iphone xr 128gb,22990000,Khách hàng chọn 1 trong 2 khuyến mại sau:KM1:T...,dienthoai
8,iphone x 64gb,22990000,Cơ hội trúng 600 Chỉ Vàng 9999 Trả góp 0% Tặng...,dienthoai
9,samsung galaxy note 9 128gb,22990000,Khách hàng chọn 1 trong 3 khuyến mại sau:KM1:T...,dienthoai


In [8]:
def links_dienthoai_tgdd(url):
    #options = webdriver.ChromeOptions()

    #options.binary_location = path_chrome
    chrome_driver_binary = path_chrome_driver

    #driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
    driver = webdriver.Chrome(chrome_driver_binary)
    driver.get(url)

    num = 0

    for i in range(2):
        for elem in driver.find_elements_by_css_selector("body > section > a"):
            driver.execute_script("javascript:More();", elem)
            sleep(3)

    # Selenium hands the page source to Beautiful Soup
    tgdd = BeautifulSoup(driver.page_source, 'lxml')
    
    links = []
    allPhone = tgdd.findAll('a', href  = re.compile("^(/dtdd/)"))
    for phone in allPhone:
        link = 'https://www.thegioididong.com' + phone.attrs['href']
        links.append(link)
    
    driver.close()
    return links

urls_tgdd = ['https://www.thegioididong.com/dtdd-apple-iphone',
        'https://www.thegioididong.com/dtdd-samsung',
        'https://www.thegioididong.com/dtdd-oppo',
        'https://www.thegioididong.com/dtdd-nokia',
        'https://www.thegioididong.com/dtdd-huawei',
        'https://www.thegioididong.com/dtdd-sony',
        'https://www.thegioididong.com/dtdd-realme',
        'https://www.thegioididong.com/dtdd-xiaomi',
        'https://www.thegioididong.com/dtdd-vivo',
        'https://www.thegioididong.com/dtdd-philips',
        'https://www.thegioididong.com/dtdd-mobell',
        'https://www.thegioididong.com/dtdd-mobiistar',
        'https://www.thegioididong.com/dtdd-itel',
        'https://www.thegioididong.com/dtdd-htc',
        'https://www.thegioididong.com/dtdd-coolpad',
        'https://www.thegioididong.com/dtdd-motorola']

link_tgdd = []
for url in urls_tgdd:
    link_tgdd.extend(links_dienthoai_tgdd(url))
    
i = 1
df_tgdd = pd.DataFrame(columns=['Name', 'Price', 'Promotion'])

names = []
prices = []
KMs = []
num = 0

try:
    for url in link_tgdd:
        response = requests.get(url)
        obj = BeautifulSoup(response.text,"lxml")    

        name = obj.find('h1')
        price = obj.find('div', {'class':'area_price'})
        KM = obj.find('div', {'class':'infopr'})
        KM1 = obj.find('div', {'class':'g_j3pro'})
        KMonline = obj.find('aside', {'class':'onlinepromo'})
        Pshock = obj.find('div', {'class':'boxshockheader'})
        
        if (price is not None):
            if(price.find('strong') is not None):
                p = price.find('strong').get_text().encode('utf-8').strip()
            else:
                p = obj.find('a', {'class':re.compile("^(item).*(active )$")})
                
                if (p is not None):
                    p = p.find('strong').get_text().encode('utf-8').strip()
                else:
                    p = ''                
        else:
            p = obj.find('a', {'class':re.compile("^(item)(active )$")})
            if (p is not None):
                p = p.find('strong').get_text().encode('utf-8').strip()
            else:
                p = ''
        
        if (p == ''):
            try:
                p = Pshock.find('label').find('strong').get_text().encode('utf-8').strip()
            except:
                p = ''
                
        if (name  is not None):
            n = name.get_text().encode('utf-8').strip()
        else:
            n = ''
        if (KM  is not None):
            pr = KM.get_text().encode('utf-8').strip()
        else:
            pr = ''

        if (KM1  is not None):
            pr1 = KM1.get_text().encode('utf-8').strip()
        else:
            pr1 = ''
        
          
        if (KMonline is not None):
            text1 = KMonline.find('b').get_text().encode('utf-8').strip()
            text2 = KMonline.find('h3')
            text3 = KMonline.find('div', {'class':'infopr'})
            if(text2 is not None):
                text2 = text2.get_text().encode('utf-8').strip()
            else:
                text2 = ''
            if(text3 is not None):
                text3 = text3.get_text().encode('utf-8').strip()
            else:
                text3 = ''

            pr2 = text1 + text2 + text3
        else:
            pr2 = ''

        pr = pr + pr1 + pr2
        
        names.append(n)
        prices.append(p)
        KMs.append(pr)

    for i in range(len(names)):
        df_tgdd.loc[i, 'Name'] = names[i].lower().strip()
        df_tgdd.loc[i, 'Price'] = prices[i]
        df_tgdd.loc[i, 'Promotion'] = KMs[i].strip()
        df_tgdd.loc[i, 'Name'] = df_tgdd.loc[i, 'Name'].replace('Điện thoại ', '')

    df_tgdd['Maingroup'] = 'dienthoai'
except:
    print 'Could not open TGDD'
    
to_number(df_tgdd)

,Name,Price,Promotion,Maingroup
0,iphone xs max 512gb,43990000,Giảm ngay 2 triệu * Giảm 2 triệu khi thanh toá...,dienthoai
1,iphone xs max 256gb,37990000,Giảm ngay 2 triệu * Giảm 2 triệu khi thanh toá...,dienthoai
2,iphone xs 256gb,34990000,Giảm ngay 2 triệu * Giảm 2 triệu khi thanh toá...,dienthoai
3,iphone xs max 64gb,33990000,Giảm ngay 2 triệu * Giảm 2 triệu khi thanh toá...,dienthoai
4,iphone xs 64gb,29990000,Giảm ngay 2 triệu * Giảm 2 triệu khi thanh toá...,dienthoai
5,iphone x 256gb,27990000,Gói bảo hành 2 năm (áp dụng đến 28/2),dienthoai
6,iphone 8 plus 256gb,,,dienthoai
7,iphone xr 256gb,24990000,Giảm 2 triệu khi thanh toán qua thẻ Mastercard...,dienthoai
8,iphone 7 plus 256gb,23990000,,dienthoai
9,iphone x 64gb,22990000,Gói bảo hành 2 năm (áp dụng đến 28/2),dienthoai


In [9]:
### SCRAPE VIENTHONGA - DIENTHOAI
link_vta_dien_thoai = "https://vienthonga.vn/dien-thoai-smartphones"
df_vta = pd.DataFrame(columns=['Name', 'Price', 'Promotion'])

#options = webdriver.ChromeOptions()

#options.binary_location = path_chrome
chrome_driver_binary = path_chrome_driver

#driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
driver = webdriver.Chrome(chrome_driver_binary)

driver.get(link_vta_dien_thoai)

links = []
num = 0

for i in range(15):
    for elem in driver.find_elements_by_css_selector("#CategoryPager > button"):
        driver.execute_script("arguments[0].click();", elem)
        sleep(4)

#Selenium hands the page source to Beautiful Soup
vta = BeautifulSoup(driver.page_source, 'lxml')

allPhone = vta.findAll('div', {'class':'product-overlay animate'})
links_vta = []

names = []
prices = []
KMs = []

if allPhone is not None:
    for phone in allPhone:
        name = phone.find('h3',{'class':'name'})
        price = phone.find('div', {'class':'overlay-price'})
        KM = phone.find('div', {'itemprop':'description'})
        link = phone.find('a')

        if (price  is not None):
            p = price.get_text().encode('utf-8')
        else:
            p = ''
        if (name  is not None):
            n = name.get_text().encode('utf-8')
        else:
            n = ''
        if (KM is not None):
            KM_detail = KM.find('div', {'style':'padding-bottom: 7px;'})
            if KM_detail is not None:
                k = KM_detail.get_text().encode('utf-8')
            else:
                k = ''
        else:
            k = ''

        names.append(n)
        prices.append(p)
        KMs.append(k)

for i in range(len(names)):
    df_vta.loc[i, 'Name'] = names[i].lower().strip()
    df_vta.loc[i, 'Price'] = prices[i]
    df_vta.loc[i, 'Promotion'] = KMs[i].strip()
    df_vta.loc[i, 'Name'] = df_vta.loc[i, 'Name'].replace('Điện thoại ', '')

df_vta['Maingroup'] = 'dienthoai'
to_number(df_vta)
driver.close()

In [20]:
df_fpt.shape, df_vta.shape, df_tgdd.shape,df_vts.shape

((195, 4), (408, 4), (209, 4), (175, 4))

In [12]:
df_vts.sort_values(by=['Name'], inplace=True)
df_fpt.sort_values(by=['Name'], inplace=True)
df_vta.sort_values(by=['Name'], inplace=True)
df_tgdd.sort_values(by=['Name'], inplace=True)

In [13]:
import xlsxwriter

# Create a workbook and add a worksheet.
filename = time.strftime("%Y%m%d-%H%M%S") + '.xlsx'

workbook = xlsxwriter.Workbook(filename)
worksheet1 = workbook.add_worksheet('fpt')
worksheet2 = workbook.add_worksheet('vts')
worksheet3 = workbook.add_worksheet('vta')
worksheet4 = workbook.add_worksheet('tgdd')

writer = pd.ExcelWriter(filename, engine='openpyxl')
df_fpt.to_excel(writer,'fpt',index=False)
df_vts.to_excel(writer,'vts',index=False)
df_vta.to_excel(writer,'vta',index=False)
df_tgdd.to_excel(writer,'tgdd',index=False)
writer.save()

In [14]:
def time_diff_str(t1, t2):
    """
    Calculates time durations.
    """
    diff = t2 - t1
    mins = int(diff / 60)
    secs = round(diff % 60, 2)
    return str(mins) + " mins and " + str(secs) + " seconds"

print " %s " % time_diff_str(t0, time.time())

 86 mins and 48.84 seconds 
